In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('DATABEFOREPERCENTILE.csv')

In [3]:
data.drop(columns=['Unnamed: 0'], inplace = True)

In [4]:
mis = data.isnull().sum()
mis

Country Name                                                                0
Time                                                                        0
Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estima

In [5]:
import pandas as pd

def load_and_preprocess_data(data):
    data['Time'] = pd.to_datetime(data['Time'], format='%Y')
    data = data.set_index('Time')
    return data

data = load_and_preprocess_data(data)

country_data = data[data['Country Name'] == 'South Africa']

In [6]:
missing = country_data.isnull().sum()
missing

Country Name                                                                0
Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estimate)         0
Net ODA received (% of GNI)                                     

In [7]:
from scipy.stats import kendalltau

def kendall_tau_test(series):
    time = np.arange(len(series))
    tau, p_value = kendalltau(time, series)
    return tau, p_value

def check_trends_kendall_tau(data, country):
    country_data = data[data['Country Name'] == country]
    indicators = country_data.columns.difference(['Country Name'])
    trend_results = {}

    for indicator in indicators:
        series = country_data[indicator]
        tau, p_value = kendall_tau_test(series)
        trend_results[indicator] = p_value < 0.1  
    
    return trend_results

trend_results = check_trends_kendall_tau(data, 'South Africa')
trend_results


{'Access to electricity (% of population)': True,
 'Agriculture, forestry, and fishing, value added (annual % growth)': False,
 'Control of Corruption: Estimate': True,
 'Employment in agriculture (% of total employment) (modeled ILO estimate)': False,
 'Employment in industry (% of total employment) (modeled ILO estimate)': True,
 'Employment in services (% of total employment) (modeled ILO estimate)': True,
 'Exports of goods and services (% of GDP)': True,
 'Foreign direct investment, net inflows (% of GDP)': False,
 'GDP (current US$)': True,
 'GDP growth (annual %)': True,
 'GDP per capita (current US$)': False,
 'GDP per capita growth (annual %)': True,
 'Government Debt to GDP': True,
 'Government Effectiveness: Estimate': True,
 'Gross capital formation (annual % growth)': True,
 'Imports of goods and services (% of GDP)': False,
 'Individuals using the Internet (% of population)': True,
 'Industry (including construction), value added (annual % growth)': True,
 'Inflation, GDP

In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression

def linear_trend_test(series):
    time = np.arange(len(series)).reshape(-1, 1)
    values = series.values.reshape(-1, 1)
    model = LinearRegression().fit(time, values)
    r_squared = model.score(time, values)
    return r_squared

def log_transform_trend_test(series):
    series_log = np.log(series)
    series_log = series_log.replace([np.inf, -np.inf], np.nan).dropna()
    return linear_trend_test(series_log)

def determine_trend_types(data, country):
    country_data = data[data['Country Name'] == country]
    indicators = country_data.columns.difference(['Country Name'])
    trend_types = {}

    for indicator in indicators:
        series = country_data[indicator].dropna()
        r_squared_linear = linear_trend_test(series)
        r_squared_log = log_transform_trend_test(series)

        if r_squared_linear > 0.5:
            trend_types[indicator] = 'additive'
        elif r_squared_log > 0.5:
            trend_types[indicator] = 'multiplicative'
        else:
            trend_types[indicator] = 'undetermined'

    return trend_types

trend_types = determine_trend_types(data, 'South Africa')
trend_types


/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/

{'Access to electricity (% of population)': 'additive',
 'Agriculture, forestry, and fishing, value added (annual % growth)': 'undetermined',
 'Control of Corruption: Estimate': 'additive',
 'Employment in agriculture (% of total employment) (modeled ILO estimate)': 'undetermined',
 'Employment in industry (% of total employment) (modeled ILO estimate)': 'additive',
 'Employment in services (% of total employment) (modeled ILO estimate)': 'additive',
 'Exports of goods and services (% of GDP)': 'undetermined',
 'Foreign direct investment, net inflows (% of GDP)': 'undetermined',
 'GDP (current US$)': 'undetermined',
 'GDP growth (annual %)': 'undetermined',
 'GDP per capita (current US$)': 'undetermined',
 'GDP per capita growth (annual %)': 'undetermined',
 'Government Debt to GDP': 'additive',
 'Government Effectiveness: Estimate': 'additive',
 'Gross capital formation (annual % growth)': 'undetermined',
 'Imports of goods and services (% of GDP)': 'undetermined',
 'Individuals using

In [9]:
from statsmodels.tsa.stattools import acf

def check_seasonality_acf(series, lags=20, threshold=0.5):
    acf_values = acf(series.dropna(), nlags=lags)
    significant_lags = [i for i, val in enumerate(acf_values) if abs(val) > threshold and i != 0]
    return significant_lags

def check_seasonality(data, country, lags=20, threshold=0.5):
    country_data = data[data['Country Name'] == country]
    indicators = country_data.columns.difference(['Country Name'])
    seasonality_results = {}

    for indicator in indicators:
        series = country_data[indicator]
        significant_lags = check_seasonality_acf(series, lags=lags, threshold=threshold)
        seasonality_results[indicator] = significant_lags
    
    return seasonality_results

seasonality_results = check_seasonality(data, 'South Africa')
seasonality_results


{'Access to electricity (% of population)': [1],
 'Agriculture, forestry, and fishing, value added (annual % growth)': [2],
 'Control of Corruption: Estimate': [1, 2],
 'Employment in agriculture (% of total employment) (modeled ILO estimate)': [1],
 'Employment in industry (% of total employment) (modeled ILO estimate)': [1,
  2],
 'Employment in services (% of total employment) (modeled ILO estimate)': [1,
  2],
 'Exports of goods and services (% of GDP)': [],
 'Foreign direct investment, net inflows (% of GDP)': [],
 'GDP (current US$)': [1],
 'GDP growth (annual %)': [],
 'GDP per capita (current US$)': [1],
 'GDP per capita growth (annual %)': [],
 'Government Debt to GDP': [1, 2, 3],
 'Government Effectiveness: Estimate': [1],
 'Gross capital formation (annual % growth)': [],
 'Imports of goods and services (% of GDP)': [],
 'Individuals using the Internet (% of population)': [1, 2, 3],
 'Industry (including construction), value added (annual % growth)': [],
 'Inflation, GDP defl

In [10]:
def summarize_trend_results(trend_results):
    trend_present = sum(trend_results.values())
    total_indicators = len(trend_results)
    trend_proportion = trend_present / total_indicators
    return trend_present, trend_proportion

trend_present, trend_proportion = summarize_trend_results(trend_results)
print(f'Trends detected in {trend_present} out of {len(trend_results)} indicators ({trend_proportion*100:.2f}%)')


Trends detected in 23 out of 31 indicators (74.19%)


In [11]:
def summarize_trend_types(trend_types):
    trend_counts = pd.Series(trend_types).value_counts()
    majority_trend_type = trend_counts.idxmax()
    return trend_counts, majority_trend_type

trend_counts, majority_trend_type = summarize_trend_types(trend_types)
print(f'Trend types count:\n{trend_counts}')
print(f'Majority trend type: {majority_trend_type}')


Trend types count:
undetermined    19
additive        12
Name: count, dtype: int64
Majority trend type: undetermined


In [12]:
def summarize_seasonality_results(seasonality_results):
    seasonality_count = sum(1 for lags in seasonality_results.values() if len(lags) > 0)
    total_indicators = len(seasonality_results)
    seasonality_proportion = seasonality_count / total_indicators
    return seasonality_count, seasonality_proportion

seasonality_count, seasonality_proportion = summarize_seasonality_results(seasonality_results)
print(f'Seasonality detected in {seasonality_count} out of {len(seasonality_results)} indicators ({seasonality_proportion*100:.2f}%)')


Seasonality detected in 17 out of 31 indicators (54.84%)
